In [2]:
from selenium import webdriver
from time import sleep
import time
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException, TimeoutException, NoSuchElementException, ElementClickInterceptedException
from wordcloud import WordCloud
import re
import matplotlib.pyplot as plt
from transformers import pipeline

In [3]:
path_to_chromedriver = r"C:\Path\To\chromedriver.exe"
chrome_binary_path = r"C:\Drivers\chrome-win64\chrome.exe"
chrome_options = webdriver.ChromeOptions()
chrome_options.binary_location = chrome_binary_path
chrome_options.add_argument('executable_path=' + path_to_chromedriver)
browser = webdriver.Chrome(options=chrome_options)
browser.get('https://www.flipkart.com/lenovo-23-8-inch-full-hd-va-panel-3-side-near-edgeless-tuv-eye-care-monitor-d24-20/p/itm8f8c6bfc86fb5?pid=MONFV5HRNF4QFVG4&lid=LSTMONFV5HRNF4QFVG4ATFEMN&marketplace=FLIPKART&store=6bo%2Fg0i%2F9no&srno=b_1_5&otracker=browse&otracker1=hp_rich_navigation_PINNED_neo%2Fmerchandising_NA_NAV_EXPANDABLE_navigationCard_cc_3_L2_view-all&fm=organic&iid=f6810f2e-df99-4107-9b26-8c4b59a9a694.MONFV5HRNF4QFVG4.SEARCH&ppt=hp&ppn=homepage&ssid=qvynpv4jpc0000001693079777287')
browser.maximize_window()

In [4]:
mydiv = browser.find_element(By.CLASS_NAME, '_3UAT2v._16PBlm')
parent = mydiv.find_element(By.XPATH, '..')
reviewlink = parent.get_attribute('href')
browser.get(reviewlink)

In [5]:
reviews = []
i=0
while True:
    try:
        if i == 0:
            review = browser.find_elements(By.CLASS_NAME,'t-ZTKy')
            for r in review:
                rt = r.text
                if len(rt.split()) > 2:
                    reviews.append(rt)
                nextreviewpage = browser.find_element(By.CLASS_NAME,"_1LKTO3")
            nextreviewpage = nextreviewpage.get_attribute('href')
            browser.get(nextreviewpage)
            i+=1
            
        if browser.find_element(By.CLASS_NAME,"_1LKTO3"):
            print(i*10)
            i+=1
            review = browser.find_elements(By.CLASS_NAME,'t-ZTKy')
            for r in review:
                rt = r.text
                if len(rt.split()) > 2:
                        reviews.append(rt)
            nextreviewpages=[]
            nextreviewpage = browser.find_elements(By.CLASS_NAME,"_1LKTO3")
            for review in nextreviewpage:
                review = review.get_attribute('href')
                nextreviewpages.append(review)
            if len(nextreviewpages)>1:
                browser.get(nextreviewpages[1])
            else:
                print('else')
                review = browser.find_elements(By.CLASS_NAME,'t-ZTKy')
                for r in review:
                    rt = r.text
                    if len(rt.split()) > 2:
                        reviews.append(rt)
                print(reviews)
                break
    except (StaleElementReferenceException, ElementClickInterceptedException, TimeoutException, NoSuchElementException):
        print(reviews)
        break

10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
else
['Slim, thin bazel and good colour contrast. Size also good for all purpose. I like it.', 'Good monitor for this price range.\n\nIt comes with,\n1. HDMI port\n2. VGA port\n3. Audi line out (3.5 mm jack)\n\nGot this item within stipulated delivery dates.', "Great picture Quality!!! satisfying all my needs like in work, entertainment and gaming !!👍\nPros:\n1.Quality of display colours, PPi everything is great for it's price\n2.Matt finish display is great and 75Hz is more than enough for all kind of tasks! If ur not a hard core gamer!!\n\nCons:\n1.only one HDMI port I wish there would be 2 HDMI ports in future models", 'Very good product\nMust buy', 'Low price, gud quality', 'Decent products in this range ...it upto 75 Harz best va pannel good monitor vga hbmi port also available', 'Extremely satisfied been using for 3 weeks', 'At first i got a damaged product but flipkart was fast

In [6]:
len(reviews)

203

In [7]:
'''from openpyxl import Workbook

workbook = Workbook()
sheet = workbook.active

sheet.cell(row=1, column=1, value='Reviews')

for i, review in enumerate(reviews, start=1):
    sheet.cell(row=i+1, column=1, value=review)

workbook.save('reviewsforpractice.xlsx')'''

"from openpyxl import Workbook\n\nworkbook = Workbook()\nsheet = workbook.active\n\nsheet.cell(row=1, column=1, value='Reviews')\n\nfor i, review in enumerate(reviews, start=1):\n    sheet.cell(row=i+1, column=1, value=review)\n\nworkbook.save('reviewsforpractice.xlsx')"

In [8]:
'''for rt in reviews:
    rt= re.sub(r'[^a-zA-Z0-9\s\.,!?]', '', rt)
all_words = ' '.join([txt for txt in reviews])
wordcloud = WordCloud(width = 500, height = 300, random_state = 21, max_font_size = 119).generate(all_words)
plt.imshow(wordcloud, interpolation = 'bilinear')
plt.axis('off')
plt.show()'''

"for rt in reviews:\n    rt= re.sub(r'[^a-zA-Z0-9\\s\\.,!?]', '', rt)\nall_words = ' '.join([txt for txt in reviews])\nwordcloud = WordCloud(width = 500, height = 300, random_state = 21, max_font_size = 119).generate(all_words)\nplt.imshow(wordcloud, interpolation = 'bilinear')\nplt.axis('off')\nplt.show()"

In [9]:
all_words = ''.join([txt for txt in reviews])
len(all_words)

all_words = all_words.replace("\n", " ")

In [11]:
import torch
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
model_name = 'tuner007/pegasus_summarizer'
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(torch_device)

def get_response(input_text):
  batch = tokenizer([input_text],truncation=True,padding='longest',max_length=1024, return_tensors="pt").to(torch_device)
  gen_out = model.generate(**batch,max_length= 100,num_beams=5, num_return_sequences=1, temperature=1.5)
  output_text = tokenizer.batch_decode(gen_out, skip_special_tokens=True)
  return output_text
get_response(all_words)

['A user wrote, "I bought this for 9,899/-. The screen quality is awesome. If you are searching for great picture quality in budget this product is for you." Another user wrote, "I bought this at 6300. Screen has already been rejected by 3-4 customers. Always buy 2k or more resolution for the money."']

In [14]:
for i in range (len(reviews)):
    if "I bought this for 9,899/-. The screen quality is awesome. If you are searching for great picture quality in budget this product is for you." in reviews:
        print(i)
    i+=1